# Bulk procesor

In [1]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import ALTITUDE_SKYDIVE_PARACLETE_XP
from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import aggregateResults
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable
from ssscoring import processAllJumpFiles
from ssscoring import totalResultsFrom
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import ipywidgets as widgets
import pandas as pd

In [2]:
DATA_LAKE_ROOT = './data'

In [3]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [4]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## Results

In [5]:
aggregate = aggregateResults(jumpResults)
sum = totalResultsFrom(aggregate)
display(aggregate)
display(sum)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
R1_12-12-32,414.558,175.896,307.368,383.148,405.612,415.584,25.0,418.032
R2_13-22-20,447.534,176.040,305.712,387.864,430.344,450.900,25.0,451.692
R3_14-29-41,451.62,175.068,307.548,397.548,442.872,452.196,25.0,453.384
R4_15-41-08,440.82,179.424,303.048,386.100,427.464,440.712,25.0,441.720
R5_17-02-21,427.176,180.432,314.964,397.476,417.924,421.704,25.0,429.768
R6_18-17-00,432.99,171.180,286.200,380.772,420.696,431.064,25.0,438.120
R7_11-49-45,440.784,178.308,310.608,397.836,440.568,415.260,25.0,446.364
R8_13-00-55,439.146,178.848,302.724,386.784,431.136,440.964,25.0,441.792


,totalSpeed,meanSpeed,maxScore
totalSpeed,3494.628,436.8285,451.62


In [6]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in jumpResults.keys():
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        graphJumpResult(initializePlot(resultRef),
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">R1_12-12-32 jump - 🟢 valid</span></h1>')

HTML(value='<h3>Max speed = 418; exit at 4198 m (13775 ft), end scoring window at 1942 m (6374 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">R2_13-22-20 jump - 🟢 valid</span></h1>')

HTML(value='<h3>Max speed = 452; exit at 4219 m (13841 ft), end scoring window at 1963 m (6440 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">R3_14-29-41 jump - 🟢 valid</span></h1>')

...

HTML(value='<h3>Max speed = 438; exit at 4126 m (13537 ft), end scoring window at 1870 m (6136 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">R7_11-49-45 jump - 🟢 valid</span></h1>')

HTML(value='<h3>Max speed = 446; exit at 4084 m (13400 ft), end scoring window at 1828 m (5998 ft)</h3>')

HTML(value='<hr><h1><span style="color: #0f0">R8_13-00-55 jump - 🟢 valid</span></h1>')

HTML(value='<h3>Max speed = 442; exit at 4166 m (13668 ft), end scoring window at 1910 m (6267 ft)</h3>')

## All jumps

In [7]:
bp.show(allJumpsPlot)